In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone -b feature/add_rl_part https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 532 (delta 86), reused 50 (delta 37), pack-reused 380 (from 3)
Receiving objects: 100% (532/532), 222.32 KiB | 8.89 MiB/s, done.
Resolving deltas: 100% (296/296), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 18.2 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=7f0cb71e2cc9884

In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import tensorflow as tf

2025-12-01 22:48:10.826922: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764629291.001832      21 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764629291.058565      21 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [5]:
from src.backtest.behavior_policy import run_behavior_policy_with_guards
from src.rl.state_builder import ensure_z_column, build_rl_dataset
from src.rl.cql_agent import CQLAgentTF, CQLConfig

In [6]:
# 1) Load final stacked dataset from 03_calibrate_and_stack
final_path = "/kaggle/input/01-prepare-data/final_dataset.csv"
df = pd.read_csv(final_path, parse_dates=["Date"])

In [7]:
# 2) Ensure we have a z column (dummy for now)
z_col ="dummy_z"
df = ensure_z_column(df, z_col=z_col, seed=42)

Using randomly generated dummy z column


In [8]:
# 3) Run dumb behavior policy + guards
df_policy = run_behavior_policy_with_guards(
    df,
    z_col="dummy_z",    # later change to "z" when you have real alpha
    nav=1_000_000.0,
    allow_short=False,
    trading_enabled=True,
)

In [9]:
# 4) Build RL training dataset (s, a, r, s')
extra_state_cols = ["has_news", "disagreement"]
rl_df = build_rl_dataset(
    df_policy,
    z_col="dummy_z",
    reward_return_col="excess_return",
    action_col="action_weight_raw",
    weight_col="weight_after_guards",
    extra_state_cols=None,  # only if those exist
    lambda_risk=0.1,
)
rl_df

,Date,ticker,reward,action,done,state_dummy_z,state_VIX_z,state_spread_z,state_weight_after_guards,next_state_dummy_z,next_state_VIX_z,next_state_spread_z,next_state_weight_after_guards
0,2010-01-04 00:00:00+00:00,AAPL,-0.000000,0.00,0,0.304717,-1.249591,-0.231379,0.000000,-1.039984,-1.314181,-0.015455,0.000000
8,2010-01-05 00:00:00+00:00,AAPL,-0.000000,0.00,0,-1.039984,-1.314181,-0.015455,0.000000,0.750451,-1.323599,0.871336,0.000000
16,2010-01-06 00:00:00+00:00,AAPL,-0.000000,0.00,0,0.750451,-1.323599,0.871336,0.000000,0.940565,-1.324017,1.082035,0.006999
24,2010-01-07 00:00:00+00:00,AAPL,0.000002,0.01,0,0.940565,-1.324017,1.082035,0.006999,-1.951035,-1.413993,0.266584,0.000000
32,2010-01-08 00:00:00+00:00,AAPL,-0.000007,0.00,0,-1.951035,-1.413993,0.266584,0.000000,-1.302180,-1.465035,0.106547,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21858,2018-12-20 00:00:00+00:00,WMT,0.000000,0.00,0,-2.315280,2.598611,3.000000,0.000000,-1.540832,2.909232,-0.975390,0.000000
21868,2018-12-21 00:00:00+00:00,WMT,0.000000,0.00,0,-1.540832,2.909232,-0.975390,0.000000,1.189034,3.000000,2.115415,0.000000
21878,2018-12-24 00:00:00+00:00,WMT,0.000000,0.02,0,1.189034,3.000000,2.115415,0.000000,0.369053,2.812017,-2.678646,0.003489
21888,2018-12-26 00:00:00+00:00,WMT,-0.000012,0.01,0,0.369053,2.812017,-2.678646,0.003489,-0.353816,2.675121,-2.617727,0.000000


In [10]:
#5) Extract state / next_state matrices and targets 
state_cols = [c for c in rl_df.columns if c.startswith("state_")]
next_state_cols = [c for c in rl_df.columns if c.startswith("next_state_")]

states = rl_df[state_cols].to_numpy().astype(np.float32)          # [N, state_dim]
next_states = rl_df[next_state_cols].to_numpy().astype(np.float32)

rewards = rl_df["reward"].to_numpy().astype(np.float32).reshape(-1, 1)  # [N,1]
dones = rl_df["done"].to_numpy().astype(np.float32).reshape(-1, 1)

# action is in weight units {0.0, 0.01, 0.02}. Map to indices {0,1,2}.
actions_w = rl_df["action"].to_numpy().astype(np.float32)
actions_idx = np.round(actions_w * 100).astype(np.int32).reshape(-1, 1)
# (0.00 -> 0, 0.01 -> 1, 0.02 -> 2)

state_dim = states.shape[1]
n_actions = 3

In [11]:
batch_size = 1024

dataset = tf.data.Dataset.from_tensor_slices(
    (states, actions_idx, rewards, next_states, dones)
)
dataset = dataset.shuffle(buffer_size=len(rl_df), reshuffle_each_iteration=True)
dataset = dataset.batch(batch_size)

I0000 00:00:1764629331.574998      21 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [12]:
#6) Training loop
config = CQLConfig(
    gamma=0.99,
    alpha=1.0,
    lr=1e-3,
    tau=0.005,
    n_actions=n_actions,
    hidden_dim=128,
)

agent = CQLAgentTF(state_dim=state_dim, config=config)

epochs = 100
steps_per_epoch = 200  # or len(rl_df) // batch_size

for epoch in range(epochs):
    logs_epoch = {"loss": 0.0, "td_loss": 0.0, "cql_loss": 0.0}
    n_batches = 0

    for step, (s_b, a_b, r_b, ns_b, d_b) in enumerate(dataset):
        metrics = agent.train_step(s_b, a_b, r_b, ns_b, d_b)
        for k in logs_epoch:
            logs_epoch[k] += float(metrics[k])
        n_batches += 1

        if step >= steps_per_epoch:
            break

    agent.update_target()

    for k in logs_epoch:
        logs_epoch[k] /= max(n_batches, 1)

    print(
        f"Epoch {epoch:03d}: "
        f"loss={logs_epoch['loss']:.6f}, "
        f"td={logs_epoch['td_loss']:.6f}, "
        f"cql={logs_epoch['cql_loss']:.6f}"
    )

Epoch 000: loss=0.927271, td=0.060383, cql=0.866889
Epoch 001: loss=0.702425, td=0.071910, cql=0.630515
Epoch 002: loss=0.559559, td=0.063336, cql=0.496223
Epoch 003: loss=0.485754, td=0.050088, cql=0.435666
Epoch 004: loss=0.457045, td=0.049134, cql=0.407911
Epoch 005: loss=0.442606, td=0.048480, cql=0.394126
Epoch 006: loss=0.435624, td=0.046409, cql=0.389215
Epoch 007: loss=0.429396, td=0.047658, cql=0.381739
Epoch 008: loss=0.427461, td=0.048596, cql=0.378865
Epoch 009: loss=0.425621, td=0.048571, cql=0.377050
Epoch 010: loss=0.422217, td=0.045118, cql=0.377099
Epoch 011: loss=0.421498, td=0.047268, cql=0.374231
Epoch 012: loss=0.418460, td=0.045830, cql=0.372629
Epoch 013: loss=0.418294, td=0.046935, cql=0.371359
Epoch 014: loss=0.417047, td=0.047842, cql=0.369205
Epoch 015: loss=0.414521, td=0.044627, cql=0.369894
Epoch 016: loss=0.415812, td=0.049062, cql=0.366749
Epoch 017: loss=0.412961, td=0.046253, cql=0.366708
Epoch 018: loss=0.413320, td=0.045794, cql=0.367526
Epoch 019: l

In [13]:
# Save the Q-network that defines the policy
agent.q.save("/kaggle/working/cql_q_policy.keras")

In [14]:
!rm -rf /kaggle/working/multimodal-eq-sizing